In [ ]:
%%writefile mapper.py
#!/usr/bin/python
import sys
import re
reload(sys)
sys.setdefaultencoding('utf-8')
for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t',1)
    except ValueError as e:
        continue
    words = re.split("\W*\s+\W*", text, flags = re.UNICODE)
    for word in words:
        print >> sys.stderr, "reporter:counter:Wiki stats,Total words,%d" % 1
        print "%s\t%d" % (word.lower(), 1)

In [ ]:
%%writefile reducer.py
#!/usr/bin/python
import sys
ckey = None
sum = 0

In [ ]:
%%writefile -a reducer.py

for line in sys.stdin:
    try:
        key,c = line.strip().split('\t',1)
        c = int(c)
    except ValueError as e:
        continue
    if ckey != key:
        if ckey:
            print "%s\t%d" % (ckey,sum)
        sum = 0
        ckey = key
    sum += count
if ckey:
    print "%s\t%d" % (ckey,sum)

In [ ]:
! hdfs dfs -ls /data/wiki

In [ ]:
%%bash

OUT_DIR="wordcount_result_"$(date +"%s%6N")
NUM_REDUCERS=8

hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming wordCount" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper.py,reducer.py \
    -mapper "python2 mapper.py" \
    -combiner "python2 reducer.py" \
    -reducer "python2 reducer.py" \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR} > /dev/null

hdfs dfs -cat ${OUT_DIR}/part-00000 | head